In [2]:
from anago.utils import load_data_and_labels
x_train, y_train = load_data_and_labels('../data/train.txt')
x_test, y_test = load_data_and_labels('../data/dev.txt')
x_train[0]

ValueError: too many values to unpack (expected 2)

这样的写法显然是不robust的，改成
```py
                line = line.split('\t')
                word, tag = line[0],line[1]
```

In [3]:
# from anago.utils import load_data_and_labels  #这里是靠空行来切割句子的，这会导致句子过于长。
import re
def load_data_and_labels(path):
    split_pattern = re.compile(r'[；;。，、？！\.\?,! ]')
    x = []
    y = []
    sentence = []
    labels = []
    with open(path,'r',encoding = 'utf8') as f:
        for line in f.readlines():#每行为一个字符和其tag，中间用tab隔开
            line = line.strip().split('\t')
            if(not line or len(line) < 2): continue
            word, tag= line[0], line[1]
            if split_pattern.match(word):
                sentence.append(word)
                labels.append(tag)
                x.append(sentence.copy())
                y.append(labels.copy())
                sentence.clear()
                labels.clear()
            else:
                sentence.append(word)
                labels.append(tag)
        if(len(sentence)):
            x.append(sentence.copy())
            sentence.clear()
            y.append(labels.copy())
            labels.clear()
    return x,y

In [4]:
x_train, y_train = load_data_and_labels('../data/train.txt')
x_test, y_test = load_data_and_labels('../data/dev.txt')

In [5]:
print(x_train[0])
print(y_train[0])

['当', '希', '望', '工', '程', '救', '助', '的', '百', '万', '儿', '童', '成', '长', '起', '来', '，']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [6]:
import anago
model = anago.Sequence()
model.fit(x_train, y_train, epochs=15)

/home/k/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Epoch 1/15
9331/9331 [==============================] - 2195s 235ms/step - loss: 7.3032
Epoch 2/15
9331/9331 [==============================] - 2347s 252ms/step - loss: 7.2532
Epoch 3/15
9331/9331 [==============================] - 2328s 249ms/step - loss: 7.2473
Epoch 4/15
9331/9331 [==============================] - 2351s 252ms/step - loss: 7.2439
Epoch 5/15
9331/9331 [==============================] - 2300s 246ms/step - loss: 7.2419
Epoch 6/15
9331/9331 [==============================] - 2226s 239ms/step - loss: 7.2405
Epoch 7/15
9331/9331 [==============================] - 2226s 239ms/step - loss: 7.2396
Epoch 8/15
9331/9331 [==============================] - 2225s 238ms/step - loss: 7.2387
Epoch 9/15
9331/9331 [==============================] - 2224s 238ms/step - loss: 7.2381
Epoch 10/15
9331/9331 [==============================] - 2220s 238ms/step - loss: 7.2378
Epoch 11/15
9331/9331 [==============================] - 2212s 237ms/step - loss: 7.2373
Epoch 12/15
9331/9331 [=======

训练过程如下： 

Epoch 1/15
9331/9331 [==============================] - 2319s 249ms/step - loss: 7.2982
Epoch 2/15
9331/9331 [==============================] - 2234s 239ms/step - loss: 7.2529
Epoch 3/15
9331/9331 [==============================] - 2236s 240ms/step - loss: 7.2469
Epoch 4/15
9331/9331 [==============================] - 2323s 249ms/step - loss: 7.2439
Epoch 5/15
8865/9331 [===========================>..] - ETA: 1:52 - loss: 7.2253

用jupyter跑感觉很慢，而且训练过程中打印会出现问题。

注意，model.score()只是返回一个F1-score，要print出来。由于没有print，曾浪费了10多个小时跑完15个epoch。

In [7]:
print(model.score(x_test, y_test)) #score一次需要6-7分钟

0.7459409594095942


一个epoch   
0.6514121589277165

In [ ]:
# 纯crf模型，窗口为3，就可以达到80多的F1-score,且训练时间比NN方法短的多，预测用时也短。